In [1]:
import sys
sys.path.append('../../')

In [2]:
import pandas as pd
import numpy as np
import data


In [3]:
samples_age = pd.read_pickle('../../data/geo-annotation/samples.age.cleaned.v1.pickle')

In [4]:
samples_tissue = pd.read_pickle('../../data/geo-annotation/samples.tissue.cleaned.v1.pickle')

In [5]:
samples_norms = pd.read_pickle('../../data/geo-annotation/samples.norms.07.res4.pickle')

In [19]:
samples_norms.shape

(544916, 2)

In [20]:
samples_norms.query("norm == 1").shape

(169884, 2)

In [6]:
samples_age.head()

,accession,characteristics,series,platform,organism,age,isfloat
22869,E-MTAB-872_0,"{'InitialTimePoint': 'differentiation', 'Organ...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0,True
22870,E-MTAB-872_1,"{'InitialTimePoint': 'differentiation', 'Organ...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0,True
22871,E-MTAB-872_2,"{'InitialTimePoint': 'differentiation', 'Organ...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0,True
22890,E-MTAB-487_0,"{'Organism': 'Homo sapiens', 'CellType': 'CD34...",[E-MTAB-487],A-GEOD-8490,Homo sapiens,0,True
22891,E-MTAB-487_1,"{'Organism': 'Homo sapiens', 'CellType': 'CD34...",[E-MTAB-487],A-GEOD-8490,Homo sapiens,0,True


In [7]:
samples_tissue.head()

,collapsed
item,
GSM1006272,BTO:0000759
E-MTAB-3099_162,BTO:0000759
GSM645939,BTO:0000855
GSM429771,BTO:0000848
E-MTAB-3732_3998,BTO:0000876


In [8]:
samples = (
    samples_age
    .set_index('accession')
    .join(samples_tissue)
)
samples.shape

(139273, 7)

In [9]:
samples.head()

,characteristics,series,platform,organism,age,isfloat,collapsed
accession,,,,,,,
E-MTAB-872_0,"{'InitialTimePoint': 'differentiation', 'Organ...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0,True,NaN
E-MTAB-872_1,"{'InitialTimePoint': 'differentiation', 'Organ...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0,True,NaN
E-MTAB-872_2,"{'InitialTimePoint': 'differentiation', 'Organ...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0,True,NaN
E-MTAB-487_0,"{'Organism': 'Homo sapiens', 'CellType': 'CD34...",[E-MTAB-487],A-GEOD-8490,Homo sapiens,0,True,BTO:0000725
E-MTAB-487_1,"{'Organism': 'Homo sapiens', 'CellType': 'CD34...",[E-MTAB-487],A-GEOD-8490,Homo sapiens,0,True,BTO:0000725


In [10]:
tissue_ontology = data.tissue_ontology()

Read obo graph
Name: 
Type: DiGraph
Number of nodes: 5659
Number of edges: 6430
Average in degree:   1.1362
Average out degree:   1.1362


In [11]:
skin_id = 'BTO:0001253'

descendants = tissue_ontology.descendants(skin_id)
map_f = tissue_ontology.to_parent_f(descendants)

samples_skin = (
    samples
    .assign(skin_tissue_id=lambda d: d['collapsed'].map(map_f))
    .dropna(subset=['skin_tissue_id'])
    .assign(age=lambda d: d['age'].astype(float))
    .query('1 < age < 80')
)
samples_skin.head()

,characteristics,series,platform,organism,age,isfloat,collapsed,skin_tissue_id
accession,,,,,,,,
E-MEXP-1130_1,"{'InitialTimePoint': ' ', 'Organism': 'Homo s...",[E-MEXP-1130],A-AFFY-33,Homo sapiens,10,True,BTO:0000667,BTO:0000404
E-MEXP-1130_6,"{'InitialTimePoint': ' ', 'Organism': 'Homo s...",[E-MEXP-1130],A-AFFY-33,Homo sapiens,10,True,BTO:0000667,BTO:0000404
E-MEXP-1130_11,"{'InitialTimePoint': ' ', 'Organism': 'Homo s...",[E-MEXP-1130],A-AFFY-33,Homo sapiens,10,True,BTO:0000667,BTO:0000404
E-MTAB-625_7,"{'Age': '18', 'Origin': 'Ghana', 'BioSourcePro...",[E-MTAB-625],A-GEOD-8490,Homo sapiens,18,True,BTO:0000404,BTO:0000404
E-MTAB-625_9,"{'Age': '18', 'Origin': 'Eritrea', 'BioSourceP...",[E-MTAB-625],A-GEOD-8490,Homo sapiens,18,True,BTO:0000404,BTO:0000404


In [12]:
samples_skin.shape

(1053, 8)

In [22]:
(
    samples_skin
    .merge(samples_norms, left_index=True, right_on='accession')
#     .query('norm == 1')
    .shape
)

(287, 10)

In [23]:
(
    samples_skin
    .skin_tissue_id
    .value_counts()
    .to_frame()
    .reset_index()
    .assign(index=lambda d: d['index'].map(tissue_ontology.name))
)

,index,0
0,epidermis,736
1,dermis,151
2,skin fibroblast,148
3,skin cancer cell,14
4,scalp,4


In [ ]:
! mkdir -p ../../data/geo-annotation/intermediate

In [ ]:
(
    samples_skin
    .assign(skin_tissue_name=lambda d: d.skin_tissue_id.map(tissue_ontology.name))
    .assign(series=lambda d: d['series'].map(','.join))
    .assign(platform=lambda d: d['platform'].map(lambda p: p.replace('A-GEOD-', 'GPL')))
    .reset_index()
    [['accession', 'characteristics', 'series', 'platform', 'organism', 'age', 'skin_tissue_id', 'skin_tissue_name']]
    .to_csv('../../data/geo-annotation/intermediate/samples.skin.age.v1.csv', index=False)
#     .head()
)

## Norms

In [ ]:
pd.read_pickle

## Посмотрим что за платформы

In [26]:
(
    samples_skin
    .assign(platform=lambda d: d['platform'].map(lambda p: p.replace('A-GEOD-', 'GPL')))
    .platform
    .value_counts()
    .to_frame()
)

,0
A-AGIL-11,530
GPL538,96
GPL8490,80
GPL570,73
GPL13534,66
GPL10558,53
A-AFFY-44,26
GPL10335,26
GPL6244,20
GPL16384,12


In [33]:
(
    samples_skin
    [samples_skin.series.map(lambda s: s == ['E-TABM-1'])]
    .reset_index()
    .accession
    .value_counts()
#     .query("platform == 'A-AGIL-11'")
#     .series
#     .map(','.join)
#     .value_counts()
)

E-TABM-1_3960    1
E-TABM-1_2620    1
E-TABM-1_1864    1
E-TABM-1_4547    1
E-TABM-1_6396    1
E-TABM-1_3908    1
E-TABM-1_1388    1
E-TABM-1_179     1
E-TABM-1_589     1
E-TABM-1_1808    1
E-TABM-1_3026    1
E-TABM-1_4380    1
E-TABM-1_1612    1
E-TABM-1_422     1
E-TABM-1_5084    1
E-TABM-1_1108    1
E-TABM-1_936     1
E-TABM-1_3461    1
E-TABM-1_4883    1
E-TABM-1_5796    1
E-TABM-1_1891    1
E-TABM-1_3376    1
E-TABM-1_1220    1
E-TABM-1_86      1
E-TABM-1_5057    1
E-TABM-1_4552    1
E-TABM-1_1178    1
E-TABM-1_2186    1
E-TABM-1_6564    1
E-TABM-1_4579    1
                ..
E-TABM-1_5728    1
E-TABM-1_1849    1
E-TABM-1_169     1
E-TABM-1_2783    1
E-TABM-1_2689    1
E-TABM-1_5755    1
E-TABM-1_5472    1
E-TABM-1_2228    1
E-TABM-1_2196    1
E-TABM-1_3530    1
E-TABM-1_2144    1
E-TABM-1_772     1
E-TABM-1_4720    1
E-TABM-1_5891    1
E-TABM-1_2868    1
E-TABM-1_4789    1
E-TABM-1_1430    1
E-TABM-1_6890    1
E-TABM-1_5419    1
E-TABM-1_6143    1
E-TABM-1_2563    1
E-TABM-1_127